# Summary

This will cover calculating a baseline, calculating correction factors, 
and downscaling. The notebook assumed that you have run the prior notebooks
and have a tile directory with the following files in it:

  - `cru_AnnualTimeSeries/cru_AnnualTimeSeries-YYYY.nc`
  - `manifest.yml`
  - `worldclim.nc`

Start by loading libraries. 

In [ ]:
# For development...
# %load_ext autoreload
# %autoreload 2

from temds import tile
from temds.datasources import crujra
from temds.datasources import worldclim

# Load the tile

This function will make an in memory `Tile` object with data from the directory passed to it.

In [ ]:
mytile = tile.Tile.tile_from_directory("working/03-tiles/H00_V08")
  

In [ ]:
for k,v in mytile.data.items():
  print(f"The tile has the key '{k}' to a variable of type {type(v)}")


In [ ]:
#start_year = mytile.data['cru_AnnualTimeSeries'].range()[0]
#end_year = mytile.data['cru_AnnualTimeSeries'].range()[-1]

start_year = 1970
end_year = start_year + 30
print(f"Start year: {start_year}, End year: {end_year}")

# Calculate the baseline

The docstring for `crujra.AnnualTimeSeries.create_climate_baseline()` is helpful:

In [ ]:
#print(crujra.AnnualTimeSeries.create_climate_baseline.__doc__)

Once the baseline is computed, it is now in memory as part of the `Tile` object. If you save the object you will additionally get a baseline.nc file and that should be added to the manifest.

In [ ]:
mytile.calculate_climate_baseline(start_year, start_year + 30, 'cru_baseline', 'cru_AnnualTimeSeries')
mytile.data['cru_baseline']

# Correction Factors

Now that we have a climate baseline calcuated, we need to calculate correction
factors. The correction factors are handled slightly differently for different types
of variables (i.e. mean for some values, sum for others). Additionally the 
variables used are named slightly differently in the baseline and reference datasets.

So we start here by making a dictionary holding this info that we can pass to 
the correction factor function.

In [ ]:
variables_cf = {
    'tmax': {'function': 'temperature', 'reference': 'tmax','baseline':'tmax', 'name': 'tmax'},
    'tmin': {'function': 'temperature', 'reference': 'tmin','baseline':'tmin', 'name': 'tmin'},
    'tavg': {'function': 'temperature', 'reference': 'tavg','baseline':'tmp', 'name': 'tavg'},
    'prec': {'function': 'precipitation', 'reference': 'prec','baseline':'pre', 'name': 'prec'},

    'vapo_pa': {
        'function': 'vapor-pressure', 
        'reference': 'vapr',
        'baseline-pres':'pres', 
        'baseline-spfh': 'spfh',
        'name': 'vapo'
        },
    'nirr': {
        'function': 'radiation', 
        'reference': 'srad',
        'baseline':'dswrf', 
        'name': 'nirr'
        
    },
    'ws': {
        'function': 'wind-speed', 
        'reference': 'wind',
        'baseline-ugrd': 'ugrd', 
        'baseline-vgrd': 'vgrd', 
        'name': 'ws'
        
    }
}

# Calculate correction factors for the variables
mytile.calculate_correction_factors('cru_baseline', 'worldclim', variables=variables_cf)

Now our tile object has a new `correction_factors` key in the data dictionary:

In [ ]:
print(mytile.data.keys())
print()
print("Correction Factors:")
print(mytile.data['correction_factors'])

# Downscaling

The downscaling resamples the data to a finer resolution and applies the 
correction factors.

In [ ]:
variables_ds = {
    'tmax': {'function': 'temperature', 'temperature': 'tmax','correction_factor':'tmax', 'name': 'tmax'},
    'tmin': {'function': 'temperature', 'temperature': 'tmin','correction_factor':'tmin', 'name': 'tmin'},
    'tavg': {'function': 'temperature', 'temperature': 'tmp','correction_factor':'tavg', 'name': 'tavg'},
    'prec': {'function': 'precipitation', 'precipitation': 'pre','correction_factor':'prec', 'name': 'prec'},
    'vapo': {'function': 'vapor-pressure', 'name': 'vapo', 'pres':'pres', 'spfh':'spfh', 'correction_factor': 'vapo' },
    'nirr': {'function':'radiation', 'name': 'nirr', 'dswrf':'dswrf','correction_factor': 'nirr'},
    'wind': {'function':'wind-speed', 'name': 'wind', 'ugrd': 'ugrd', 'vgrd':'vgrd', 'correction_factor':'ws'},
    'winddir': {'function':'wind-direction', 'name': 'winddir', 'ugrd': 'ugrd', 'vgrd':'vgrd' },
}

mytile.downscale_timeseries('downscaled_cru', 'cru_AnnualTimeSeries','correction_factors', variables_ds, True)




Once we have the downscaling done, we can save the data for the tile.



In [ ]:
# Make sure the output directory exists
!mkdir -p working/04-downscaled-tiles/
mytile.save("working/04-downscaled-tiles", items=['downscaled_cru', 'cru_baseline', 'correction_factors'], complevel=1)

In [ ]:
!ls working/04-downscaled-tiles/H00_V08/

# V ---- SCRATCH ---- V

In [ ]:
type(mytile.crs)

import pyproj

pyproj.CRS.from_user_input(mytile.crs).to_wkt()